In [1]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files=['sampledata/covid_abstracts.csv'])

Using custom data configuration default-72dfbc83959ebf5d
Found cached dataset csv (/Users/olang/.cache/huggingface/datasets/csv/default-72dfbc83959ebf5d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
dataset = dataset['train']
dataset

Dataset({
    features: ['title', 'abstract', 'url'],
    num_rows: 10000
})

In [3]:
def lenth_of_title(example):
    return len(example['title'])

In [4]:
dataset = dataset.map(lambda example: {'title_length': lenth_of_title(example)})
dataset

Loading cached processed dataset at /Users/olang/.cache/huggingface/datasets/csv/default-72dfbc83959ebf5d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-695ce047d55347a7.arrow


Dataset({
    features: ['title', 'abstract', 'url', 'title_length'],
    num_rows: 10000
})

In [5]:
dataset = dataset.filter(lambda example: example['title_length'] > 10)

Loading cached processed dataset at /Users/olang/.cache/huggingface/datasets/csv/default-72dfbc83959ebf5d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0d0b71090d156c4b.arrow


In [6]:
# delete url and abstract
# dataset = dataset.map(lambda example: {'title': example['title']})

In [6]:
dataset['title'][0]

'Real-World Experience with COVID-19  Including Direct COVID-19 Antigen Testing and Monoclonal-Antibody Bamlanivimab in a Rural Critical Access Hospital in South Dakota'

In [8]:
#save dataset
dataset.save_to_disk('sampledata/covid_abstracts_filtered')

Loading cached processed dataset at /Users/olang/.cache/huggingface/datasets/csv/default-72dfbc83959ebf5d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-6d489fb6e05cd5c8.arrow


In [9]:
from datasets import load_from_disk

#load dataset
dataset2 = load_from_disk('sampledata/covid_abstracts_filtered')
dataset2

Dataset({
    features: ['title', 'abstract', 'url', 'title_length'],
    num_rows: 10000
})

In [10]:
from datasets import load_from_disk as lds

d2 = lds('embeddings')
d2

Dataset({
    features: ['title', 'abstract', 'url', 'title_length', 'embedding'],
    num_rows: 10000
})

In [12]:
d2['title'][0]
#d2['embedding'][0]

'Real-World Experience with COVID-19  Including Direct COVID-19 Antigen Testing and Monoclonal-Antibody Bamlanivimab in a Rural Critical Access Hospital in South Dakota'

---

In [17]:
from transformers import AutoTokenizer, TFAutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFMPNetModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFMPNetModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFMPNetModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFMPNetModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMPNetModel for predictions without further training.


In [18]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [19]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="tf"
    )
    encoded_input = {k: v for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [20]:
embedding = get_embeddings(d2["title"][0])
embedding.shape

TensorShape([1, 768])

In [13]:
#embedding_dataset = dataset.map(lambda example: {'embedding': get_embeddings(example['title']).numpy()[0]})

In [21]:
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).numpy()
question_embedding.shape

(1, 768)

In [22]:
scores, samples = d2.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

MissingIndex: Index with index_name 'embeddings' not initialized yet. Please make sure that you call `add_faiss_index` or `add_elasticsearch_index` first.